In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('../data/qa_dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [9]:
def tokenize(sentence):
    sentence = sentence.lower()
    sentence = sentence.replace('?', '')
    sentence = sentence.replace("'", "")
    return sentence.split(' ')

In [10]:
tokenize(df['question'][0])

['what', 'is', 'the', 'capital', 'of', 'france']

In [18]:
def build_vocab(row):
    merged = tokenize(row['question']) + tokenize(row['answer'])
    for word in merged:
        if word not in vocab:
            vocab[word] = len(vocab)

In [20]:
vocab = {"<OOV>": 0}
df.apply(build_vocab, axis = 1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [29]:
def sentence_to_indices(sentence, vocab):
    sequence = []
    for word in tokenize(sentence):
        if word in vocab:
            sequence.append(vocab[word])
        else:
            sequence.append(0)
    return sequence

In [30]:
sentence_to_indices(df['question'][0], vocab)

[1, 2, 3, 4, 5, 6]

In [39]:
import torch
from torch.utils.data import Dataset, DataLoader

class QADataset(Dataset):
    def __init__(self, df, vocab):
        super().__init__()
        self.data = df
        self.vocab = vocab

    def __len__(self):
        return self.data.shape[1]

    def __getitem__(self, index):
        features = torch.tensor(sentence_to_indices(df['question'][index], self.vocab))
        label = torch.tensor(sentence_to_indices(df['answer'][index], self.vocab))
        return features, label

In [41]:
dataset = QADataset(df, vocab)
dataloader = DataLoader(dataset=dataset, batch_size=1, shuffle=True)

In [65]:
import torch.nn as nn

class SimpleRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim=embed_dim)
        self.rnn = nn.RNN(embed_dim, 64)
        self.fc = nn.Linear(64, vocab_size)

    def forward(self, question):
        embedded_question = self.embedding(question)
        hidden, final = self.rnn(embedded_question)
        output = self.fc(final)

        return output
        

In [66]:
learning_rate = 0.001
epochs = 20

In [67]:
model = SimpleRNN(len(vocab), 50)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

In [70]:
for epoch in range(epochs):

    total_epoch_loss = 0
    
    for bfeatures, blabels in dataloader:

        optimizer.zero_grad()

        output = model(bfeatures)

        loss = loss_function(output, blabels)

        loss.backwards()
        
        optimizer.step()

        total_epoch_loss += loss.item()

    print(f"epoch {epoch + 1} | loss {total_epoch_loss / len(dataloader)}")

RuntimeError: Expected target size [1, 324], got [1, 1]

In [55]:
b = y(a)

In [58]:
b[0].shape

torch.Size([6, 64])

In [59]:
b[1].shape

torch.Size([1, 64])

In [60]:
z = nn.Linear(64, vocab_size)

In [61]:
c = z(b[1])

In [62]:
c.shape

torch.Size([1, 324])